In [ ]:
!pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## Caricamento dati

In [ ]:
dataset = pd.read_csv("Placement_Data_Full_Class.train.csv")

## Controllo se ci sono valori nulli ed eventualmente si sostituisco con valori "0"

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset['salary'] = dataset['salary'].fillna(0)

In [ ]:
dataset

## Individuo le variabili numeriche

In [ ]:
features_numeriche = dataset.select_dtypes(include=np.number)
features_numeriche

## Individuo le variabili categoriche

In [ ]:
features_categoriche = dataset.select_dtypes(include='object')
features_categoriche

## Uso LabelEncoder per trasformare i valori da categorico a numerico

In [ ]:
label_encoder = LabelEncoder()
for i in features_categoriche:
    features_categoriche[i] = label_encoder.fit_transform(features_categoriche[i])
features_categoriche

## Concateno i dati

In [ ]:
fullDataset = pd.concat([features_numeriche,features_categoriche], axis = 1)
fullDataset

## Separo il target dal resto dei dati

In [ ]:
y = fullDataset['status']
x = fullDataset.drop(columns=['status'])
y

In [ ]:
x

## Uso SelectFromModel per selezionare alcune features in base a un peso d'importanza, usando uno stimatore. In tal caso ho scelto di usare Ridge, utile a prevenire l'overfitting del modello.

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Ridge

In [ ]:
sfm = SelectFromModel(Ridge(max_iter=1000000,alpha=0.001))
sfm.fit(x,y)
sfm.get_support()
selected_Ridge = x.columns[(sfm.get_support())]

In [ ]:
selected_Ridge

In [ ]:
x = x[selected_Ridge]
x

## Definisco una funzione per separare le features selezionate da Ridge

In [ ]:
def transf(dataset):
    dataset = dataset.copy()
    dataset = dataset.drop(columns=['gender','ssc_p','ssc_b','hsc_p','hsc_s','degree_p','etest_p','specialisation','salary'])
    return dataset

## Carico i dati di training e di test

In [ ]:
train_csv = pd.read_csv('train.csv')
test_csv = pd.read_csv('test.csv')

## A questo punto utilizzo solo le features selezionate dallo stimatore sui dati di training e test del Notebook 1

In [ ]:
train = transf(train_csv.copy())
test = transf(test_csv.copy())

In [ ]:
train

In [ ]:
test

## Effettuo il label encoding sui set di training e test

In [ ]:
train['status'] = label_encoder.fit_transform(train['status'])
train['hsc_b'] = label_encoder.fit_transform(train['hsc_b'])
train['degree_t'] = label_encoder.fit_transform(train['degree_t'])
train['workex'] = label_encoder.fit_transform(train['workex'])

train

In [ ]:
test['status'] = label_encoder.fit_transform(test['status'])
test['hsc_b'] = label_encoder.fit_transform(test['hsc_b'])
test['degree_t'] = label_encoder.fit_transform(test['degree_t'])
test['workex'] = label_encoder.fit_transform(test['workex'])

test

## Uso DecisionTreeClassifier, ovvero un classificatore che prende in input un insieme di features e restituisce in output una decisione, e poi calcolo l'accuracy

In [ ]:
y_train, y_test = train['status'], test['status']
x_train, x_test = train.drop(columns=['status']), test.drop(columns=['status'])

tr = DecisionTreeClassifier(criterion='entropy', random_state=0, max_depth=6)

tr.fit(x_train, y_train)

In [ ]:
pred = tr.predict(x_test)
print('Accuracy:', accuracy_score(y_test,pred))